[Reference](https://luis-sena.medium.com/build-an-auto-scaling-transcoding-platform-using-python-ffmpeg-aws-950645fe5e07)

![architecture](https://miro.medium.com/max/322/1*SjebMRrURGwdTIl8VbjtiA.png)"

In [1]:
import json
import logging
import os
import subprocess

import boto3
import flask
from flask import request, Response

TRANSCODE_BUCKET_NAME = '<set your destination bucket name>'

def upload_folder(src_directory, target_s3_directory):
    s3 = boto3.client('s3')
    for root, dirs, files in os.walk(src_directory):
        for filename in files:
            local_path = os.path.join(root, filename)

            relative_path = os.path.relpath(local_path, src_directory)
            s3_path = os.path.join(target_s3_directory, relative_path)
            s3.upload_file(local_path, TRANSCODE_BUCKET_NAME, s3_path)


# Create and configure the Flask app
app = flask.Flask(__name__)


@app.route('/transcode-video', methods=['POST'])
def customer_registered():
    """transcode video"""

    response = None
    if request.json is None:
        # Expect application/json request
        response = Response("", status=415)
    else:
        try:
            if request.json.has_key('TopicArn') and request.json.has_key('Message'):
                message = json.loads(request.json['Message'])
            else:
                message = request.json

            # download from s3
            s3 = boto3.client('s3')
            download_path = '<set your download path>'
            s3.download(message['file_path'], download_path)
            HLS_PATH = 'target_hls'

            # FFmpeg transcode command
            cmd = f'ffmpeg -i {download_path} -filter_complex "[v:0]split=3[vout001][vout002][vout003]" \
                      -preset ultrafast -g 25 -sc_threshold 0 \
                      -map [vout001] -s:v:0 1280x720 -c:v:0 libx264 -b:v:0 2600k \
                      -map [vout002] -s:v:1 854x480 -c:v:1 libx264 -b:v:1 1600k \
                      -map [vout003] -s:v:2 640x360 -c:v:2 libx264 -b:v:2 1024k \
                      -map a:0 -map a:0 -map a:0 -c:a aac -b:a 128k -ac 2 \
                      -f hls -hls_time 4 -hls_playlist_type vod \
                      -master_pl_name index.m3u8 \
                      -strftime 1 -hls_segment_filename {HLS_PATH}/version_%v/data-%Y%m%d-%s.ts \
                      -use_localtime_mkdir 1 \
                      -max_muxing_queue_size 9999 \
                      -var_stream_map "v:0,a:0 v:1,a:1 v:2,a:2" {HLS_PATH}/version_%v.m3u8'

            # call FFmpeg directly
            process = subprocess.Popen(cmd,
                                       stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                       universal_newlines=True, shell=True, start_new_session=True
                                       )
            # wait for FFmpeg to finish
            process.communicate()
            upload_folder(HLS_PATH, HLS_PATH)
            response = Response("", status=200)
        except Exception as ex:
            logging.exception('Error processing message: %s' % request.json)
            response = Response(str(ex), status=500)

    return response


if __name__ == '__main__':
    app.run(host='0.0.0.0')